In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import pickle
import os
import pandas as pd
import numpy as np
print(tf.__version__)

In [ ]:
elmo = hub.Module("https://tfhub.dev/google/elmo/3")

# signature default is basically not tokenized
# embeddings = elmo(['Emily ate a lot of pi'], signature="default", as_dict=True)
# default is basically the sum of the vectors
# embeddings['default']

In [ ]:
# signature default is basically not tokenized
embeddings = elmo(['Emily ate a lot of pi'], signature="default", as_dict=True)
# default is basically the sum of the vectors
# embeddings['default']

In [ ]:
def elmo_vectors(x):
    embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.tables_initializer())
        # return average of ELMo features
        return sess.run(tf.reduce_mean(embeddings,1))

In [ ]:
a = tf.constant(5.0)
b = tf.constant(6.0)
c = a * b
sess = tf.compat.v1.keras.backend.get_session()

sess.run(c)

In [ ]:
elmo_vectors(embeddings['default'])

In [6]:
nyt_df = pd.DataFrame(pd.read_pickle('../../NYTcorpus_updated.p'))

In [9]:
nyt_df = pd.DataFrame(nyt_df)

In [11]:
nyt_df[2]

0          LEAD: *3*** COMPANY REPORTS ** *3*MARSH & MCLE...
1          LEAD: *3*** COMPANY REPORTS ** *3*NORTHERN IND...
2          LEAD: Cadbury Schweppes P.L.C., the soft-drink...
3          LEAD: *3*** COMPANY REPORTS ** *3*GENERAL EMPL...
4          LEAD: *3*** COMPANY REPORTS ** *3*POTLATCH COR...
                                 ...                        
1855653                                                     
1855654    Many American food products are a mix of globa...
1855655    Even considering the odd manner in which polit...
1855656    The preacher at the main mosque here in the Pa...
1855657    A year after President Bush and Secretary of S...
Name: 2, Length: 1855658, dtype: object

In [12]:
nyt_embeddings = elmo(nyt_df[2], signature="default", as_dict=True)

TypeError: 'AutoTrackable' object is not callable

In [ ]:
corpusdir="../", 
# corpusfile="NYTcorpus_updated.p"

# load the whole corpus
# with open(corpusdir + corpusfile, "rb") as corpus:
with open("../NYTcorpus.p", "rb") as corpus:
    all_data = pickle.load(corpus)
    print("Loaded corpus!\n")
    data = []
    labels = []
    l = len(all_data)
    for i in range(0, l): # memory errors rip
        article = all_data[i][2]
        print(article)
        # # tokenize each article
        # data.append(TaggedDocument(list(tokenize(article)), [i])) # article text
        # labels.append(all_data[i][3:]) # article labels

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
def embed_p(corpusdir="../../", 
            corpusfile="NYTcorpus_updated.p", 
            logfile="log.txt", 
            outfile="embedded_100%.p", 
            labelfile="binarylabels_100%.p", 
            labeldictfile="labelsdict_100%.p"):
    # embed each article in a pickle format corpus into a singular vector using vocabfile as the vocabulary
    # the format of the corpus is assumed to be the same as NYTcorpus.p

    f = open(logfile, 'a+')
    f.write("DOC2VEC_EMBED_P\n")
    try:
        # load the whole corpus
        with open(corpusdir + corpusfile, "rb") as corpus:
            all_data = pickle.load(corpus)
            f.write("Loaded corpus!\n")
            data = []
            labels = []
            l = len(all_data)
            for i in range(0, l): # memory errors rip
                article = all_data[i][2]
                # tokenize each article
                data.append(TaggedDocument(list(tokenize(article)), [i])) # article text
                labels.append(all_data[i][3:]) # article labels
            label_matrix, label_dict = transform_labels(labels) # make labels binary
        f.write("Words are tokenized\n")

        # transform data by doc2vec
        model = Doc2Vec(data, vector_size=100, window=5, workers=12, epochs=10)
        model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
        vec_data = []
        for i in range(0, l):
            vec_data.append(model.docvecs[i])

        # transform the test data by tf_idf as well
        f.write("Doc2Vec done\n")

        # save the results into pickle files
        with open(outfile, "wb") as out:
            pickle.dump(vec_data, out)
            f.write("Dumped data output at " + outfile + "\n")
        
        with open(labelfile, "wb") as label:
            pickle.dump(label_matrix, label)
            f.write("Dumped label output at " + labelfile + "\n")

        with open(labeldictfile, "wb") as labeldict:
            pickle.dump(label_dict, labeldict)
            f.write("Dumped label dictionary at " + labeldictfile + "\n")
        
    except Exception:
        traceback.print_exc(file=f)

    f.close()